In [1]:
from configparser import ConfigParser
import logging
import os
from pprint import pprint
import sys

In [2]:
## constants ##

SEQUENCE_PATH = "MOT20/train/MOT20-01/"
CONFIG_FILENAME = "seqinfo.ini"
DETECTIONS_DIR = "det"
DETECTIONS_FILENAME = "det.txt"

In [3]:
## logging ##

log = logging.getLogger(__name__)
out_hdlr = logging.StreamHandler(sys.stdout)
out_hdlr.setFormatter(logging.Formatter('%(asctime)s %(message)s'))
out_hdlr.setLevel(logging.DEBUG)
log.addHandler(out_hdlr)
log.setLevel(logging.DEBUG)

In [4]:
def stabilize(sequence):
    return None

In [5]:
class Sequence(object):
    
    ## ImageInfo ##
    
    class ImageInfo(object):
        def __init__(self, img_dir, filename, height, width):
            self.img_dir = img_dir
            self.filename = filename
            self.path = os.path.join(img_dir, filename)
            self.frame_id = int('.'.join(filename.split('.')[:-1]))
            self.height = height
            self.width = width

        def __str__(self):
            return "Frame: %s\tHeight: %s\t Width: %s" % (
                self.frame_id, self.height, self.width
            )

        def __repr__(self):
            return "ImageInfo(%s, %s, %s, %s)" % (
                self.img_dir, self.filename, self.height, self.width
            )
    
    ## Detection ##
    
    class Detection(object):
        def __init__(self):
            pass
    
    ## Sequence ##
    
    def __init__(self, sequence_path, config_filename, detections_dir=None, detections_filename=None):
        self.root_path = sequence_path
        self.config_filename = config_filename
        self.config_path = os.path.join(sequence_path,config_filename)        
        self.name = None
        self.length = None
        self.frame_rate = None
        self.height = None
        self.width = None
        self.image_path = None
        self._image_infos = None
        self._pos = None
        self._load_conf()
        self._load_image_infos()
        self._detections = None
        
        if detections_dir and detections_filename:
            self._load_detections()
        else:
            log.info("No detections specified, loading only images.")

    def _load_conf(self):
        
        log.info("Loading sequence configuration.")
        
        if not os.path.isfile(self.config_path):
            raise Exception("Config file, %s, doesn't seem to exist." % self.config_path)
        
        config = ConfigParser()
        config.read(self.config_path)
        sequence_config = dict(config['Sequence'])
        
        self.name = sequence_config['name']
        self.length = int(sequence_config['seqlength'])
        self.frame_rate = sequence_config['framerate']
        self.height = sequence_config['imheight']
        self.width = sequence_config['imwidth']
        self.image_path = os.path.join(self.root_path, sequence_config['imdir'])
        self.image_ext = sequence_config['imext'].replace('.','')
        
    def _load_image_infos(self):
        
        log.info("Loading image info.")
        
        dir_contents = os.listdir(self.image_path)
        file_type = self.image_ext
        image_infos = [Sequence.ImageInfo(self.image_path, x, self.height, self.width) 
                       for x in dir_contents if x.split('.')[-1] == file_type]
        
        image_infos.sort(key=lambda x: x.frame_id)
        
        if len(image_infos) != self.length:
            msg = ("The sequence length: %s, and image count: %s, do not match." % (
                len(image_infos),
                self.length
            ))
            log.debug(msg)
        
        self._image_infos= image_infos

    def _load_detections(self):
        
        log.info("Loading detections.")

    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()
    
    def next(self):
        while True:
            if self._pos is None:
                self._pos = 0
            elif self._pos < len(self._image_infos):
                current, self._pos = self._pos, self._pos + 1
                current_info = self._image_infos[current]

#                 detection_boxes = []

#                 for i, detection in current_frame.iterrows():                
#                     frame_no, traj_no, x, y, w, h, conf = detection
#                     box = Box(x, y, w, h, int(i), int(frame_no), -1, conf)
#                     detection_boxes.append(box)

#                 return Frame(current, detection_boxes)
                return current_info
            else:
                raise StopIteration()

    def __str__(self):
        return("Sequence: %s\nPath: %s\nImage Path: %s\nResolution: %sx%s\nFrame Rate: %s\nLength: %s\nType: %s"
          % (
              self.name,
              self.root_path,              
              self.image_path,
              self.width,
              self.height,
              self.frame_rate,
              self.length,
              self.image_ext
          ))
    
    def __repr__(self):
        return("Sequence(%s, %s)" % (
            self.root_path,
            self.config_filename
        ))

In [6]:
def main(argv=None):
    # arg parse blablabla    
    # if args contain config path then set it otherwise use default
    sequence_path = SEQUENCE_PATH
    config_filename = CONFIG_FILENAME
    detections_dir = DETECTIONS_DIR
    detections_filename = DETECTIONS_FILENAME
    
    sequence = Sequence(sequence_path, config_filename, detections_dir, detections_filename)
    
#     for ii in sequence:
#         print(ii)

main()

2020-05-04 13:02:39,720 Loading sequence configuration.
2020-05-04 13:02:39,724 Loading image info.
2020-05-04 13:02:39,727 Loading detections.
